# Leçon 3 : La mémoire du modèle

## Le problème de la mémoire courte

Dans les leçons précédentes, notre modèle ne regardait que la **dernière lettre**.
C'est comme essayer de deviner la fin d'une phrase en n'écoutant que le dernier mot.

Exemple : après la lettre 'a', le modèle ne sait pas si on est dans
"**cla**ra" ou "**a**dam" -- pourtant la suite est très différente !

Solution : donner une **mémoire** au modèle. On appelle ça les **embeddings**.

## Les embeddings : transformer des lettres en nombres

L'idée : chaque lettre est représentée par une **liste de nombres** (un vecteur).

Par exemple :
- 'a' -> [0.3, -0.1, 0.8]
- 'b' -> [-0.5, 0.4, 0.2]

Ces nombres ne sont pas choisis à la main : le modèle les **apprend** pendant
l'entraînement. Les lettres qui se comportent de façon similaire auront
des nombres proches.

In [ ]:
import random
import math

# Notre alphabet
alphabet = list(".abcdefghijklmnopqrstuvwxyz")
char_to_id = {c: i for i, c in enumerate(alphabet)}
id_to_char = {i: c for i, c in enumerate(alphabet)}
vocab_size = len(alphabet)

print(f"Taille du vocabulaire : {vocab_size} caractères")
print(f"Exemples : 'a' = {char_to_id['a']}, 'z' = {char_to_id['z']}, '.' = {char_to_id['.']}")

In [ ]:
# Créons les embeddings : chaque lettre = un vecteur de taille 8
EMBED_DIM = 8

random.seed(42)

# Initialisation aléatoire (le modèle apprendra les bonnes valeurs)
embeddings = [
    [random.gauss(0, 0.5) for _ in range(EMBED_DIM)]
    for _ in range(vocab_size)
]

print(f"Embedding de 'a' : {[f'{x:.2f}' for x in embeddings[char_to_id['a']]]}")
print(f"Embedding de 'b' : {[f'{x:.2f}' for x in embeddings[char_to_id['b']]]}")
print()
print("Pour l'instant ces nombres sont aléatoires.")
print("Après entraînement, les lettres similaires auront des vecteurs proches.")

## Regarder plusieurs lettres en arrière

Maintenant, au lieu de regarder 1 seule lettre, on va regarder les
**3 dernières lettres** (notre "fenêtre de contexte").

On **concatène** (met bout à bout) leurs embeddings pour avoir une image
complète du contexte.

In [ ]:
CONTEXT_SIZE = 3  # On regarde 3 lettres en arrière

def get_context_vector(mot, position, embeddings):
    """Récupère les embeddings des 3 dernières lettres et les concatène."""
    vecteur = []
    for i in range(CONTEXT_SIZE):
        pos = position - CONTEXT_SIZE + i
        if pos < 0:
            # Avant le début du mot, on utilise le padding (.)
            char_id = char_to_id['.']
        else:
            char_id = char_to_id[mot[pos]]
        vecteur.extend(embeddings[char_id])
    return vecteur

# Exemple : pour prédire la 4e lettre de "hugo"
mot = ".hugo."
position = 4  # on veut prédire 'o' (position 4)
contexte = get_context_vector(mot, position, embeddings)

print(f"Mot : '{mot}'")
print(f"Pour prédire la lettre en position {position} ('{mot[position]}'),")
print(f"on regarde les 3 lettres précédentes : '{mot[position-3:position]}'")
print(f"Vecteur de contexte : {len(contexte)} nombres ({CONTEXT_SIZE} x {EMBED_DIM})")

## Un mini réseau de neurones

On prend le vecteur de contexte et on le passe dans un **réseau de neurones**
simple (une seule couche) pour obtenir les probabilités de chaque lettre.

```
[contexte: 24 nombres] --> [couche: multiplication + addition] --> [27 scores] --> [probas]
```

In [ ]:
INPUT_DIM = CONTEXT_SIZE * EMBED_DIM  # 3 * 8 = 24

# Les poids de notre couche (une matrice 24 x 27)
W = [[random.gauss(0, 0.3) for _ in range(vocab_size)] for _ in range(INPUT_DIM)]
b = [0.0] * vocab_size  # biais

def forward(contexte, W, b):
    """Passe le contexte dans le réseau pour obtenir des scores."""
    scores = list(b)  # copie du biais
    for j in range(vocab_size):
        for i in range(INPUT_DIM):
            scores[j] += contexte[i] * W[i][j]
    return scores

def softmax(scores):
    """Transforme les scores en probabilités (entre 0 et 1, somme = 1)."""
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]

# Test
scores = forward(contexte, W, b)
probas = softmax(scores)

# Top 5 prédictions pour la lettre après 'hug' dans 'hugo'
top5 = sorted(range(vocab_size), key=lambda i: -probas[i])[:5]
print("Prédictions (avant entraînement) pour la lettre après 'hug' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print("\n  (C'est du hasard pour l'instant - il faut entraîner !)")

In [ ]:
# Entraînement
prenoms = [
    "emma", "lucas", "lea", "hugo", "chloe",
    "louis", "alice", "jules", "lina", "adam",
    "rose", "arthur", "manon", "paul", "jade",
    "nathan", "eva", "leo", "clara", "noah",
]

vitesse = 0.01

print("Entraînement avec contexte de 3 lettres...")
print()

for epoch in range(100):
    loss_totale = 0
    nb = 0

    for prenom in prenoms:
        mot = "." + prenom + "."
        for pos in range(1, len(mot)):
            cible = char_to_id[mot[pos]]

            # Forward
            ctx = get_context_vector(mot, pos, embeddings)
            scores = forward(ctx, W, b)
            probas = softmax(scores)

            # Loss
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1

            # Gradient simplifié pour W et b
            for j in range(vocab_size):
                grad = probas[j] - (1 if j == cible else 0)
                b[j] -= vitesse * grad
                for i in range(INPUT_DIM):
                    W[i][j] -= vitesse * grad * ctx[i]

    if epoch % 20 == 0:
        print(f"  Epoch {epoch:3d} | Loss : {loss_totale / nb:.3f}")

print(f"  Epoch {epoch:3d} | Loss : {loss_totale / nb:.3f}")

In [ ]:
# Générer avec le modèle entraîné
def generer(n=10):
    resultats = []
    for _ in range(n):
        mot = "."
        for _ in range(20):
            ctx = get_context_vector(mot, len(mot), embeddings)
            scores = forward(ctx, W, b)
            probas = softmax(scores)
            idx = random.choices(range(vocab_size), weights=probas, k=1)[0]
            if idx == char_to_id['.']:
                break
            mot += id_to_char[idx]
        if len(mot) > 1:
            resultats.append(mot[1:].capitalize())
    return resultats

print("Prénoms générés (avec contexte de 3 lettres) :")
print()
for p in generer(10):
    print(f"  {p}")

print()
print("Mieux qu'avant ! Le modèle 'comprend' des combinaisons de lettres.")

## Ce qu'on a appris

- Les **embeddings** transforment des lettres en nombres que le modèle peut manipuler
- Un **contexte** plus large (3 lettres au lieu de 1) donne de meilleurs résultats
- Un **réseau de neurones** (même simple) combine le contexte pour faire des prédictions

### Et ensuite ?

Notre modèle regarde toujours une fenêtre fixe de 3 lettres. Et s'il pouvait
**choisir** quelles lettres sont importantes, même si elles sont loin ?
C'est exactement ce que fait le **mécanisme d'attention** -- le cœur des GPT.

---
*Prochaine leçon : [04 - L'attention](04_lattention.ipynb)*

---

### Sources (ISO 42001)

- **Embeddings et réseau feed-forward** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy, section token/position embeddings
- **Architecture du contexte par concaténation** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023)
- **Concept d'embedding spaces** : [3Blue1Brown - Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) — Grant Sanderson